In [24]:
#imports
import pyvista as pv
import numpy as np 
import os
import mne
import pyvista
pyvista.set_jupyter_backend('static')
import pandas as pd 
import pickle
import seaborn as sns
import scipy.stats as stats
import scipy.io as sio
import random as rd 
import pyvista as pv 
import matplotlib.pyplot as plt 
from scipy.spatial import distance
from nilearn.datasets import fetch_surf_fsaverage
from nilearn import surface

# Utils import
from utils.mathutils import compute_vertex_normals, build_template_adjacency_two_hemis, compute_mean_curvature, compute_curvature_differences, compute_hausdorff_metrics, compute_point_distances, compute_normal_differences
from utils.file_manip.Matlab_to_array import load_faces, load_vertices
from utils.cortical import surface_preprocess as sp
from utils.cortical import spherical_harmonics as SH
from utils.mathutils import cart_to_sph
from utils.file_manip.vtk_processing import convert_triangles_to_pyvista

In [13]:
subject_folder = r"C:\Users\wbou2\Desktop\meg_to_surface_ml\data\Anatomy_data_CAM_CAN\sub-CC721648"
faces_file=os.path.join(subject_folder, "faces_reduced.mat")
vertices_file=os.path.join(subject_folder, "vertices_reduced.mat")
faces = load_faces(faces_file)
vertices = load_vertices(vertices_file)

In [12]:
subject_folder = r"C:\Users\wbou2\Desktop\meg_to_surface_ml\data\Anatomy_data_CAM_CAN\sub-CC721648"
faces_file=os.path.join(subject_folder, "faces_reduced.mat")
vertices_file=os.path.join(subject_folder, "vertices_reduced.mat")
faces = load_faces(faces_file)
vertices = load_vertices(vertices_file)

# Visualisation avec PyVista
mesh = pv.PolyData(vertices, convert_triangles_to_pyvista(faces))
p = pv.Plotter()
p.add_mesh(mesh, show_edges=True, color='white')
p.show()

Widget(value='<iframe src="http://localhost:34142/index.html?ui=P_0x1d2e852fa70_1&reconnect=auto" class="pyvis…

In [14]:
# Vérification des dimensions de vertices et faces
print("Vertices shape:", vertices.shape)  # Doit être (N, 3)
print("Faces shape:", faces.shape)        # Doit être (M, 3)

# Vérification des indices dans faces
if len(faces) > 0:
    print("Max face index:", np.max(faces))
    print("Vertex count:", len(vertices))
    if np.max(faces) >= len(vertices):
        print("Attention : Les indices des faces dépassent le nombre de sommets !")


Vertices shape: (15003, 3)
Faces shape: (29998, 3)
Max face index: 15002
Vertex count: 15003


In [32]:
import mne
import numpy as np

# Création du source space 
vertices = np.array(vertices, dtype=np.float64)
faces = np.array(faces, dtype=np.int32)

src_dict = {
   'rr': vertices,          
   'tris': faces,           
   'ntri': len(faces),      
   'np': len(vertices),     
   'coord_frame': 4,        
   'type': 'surf',
   'id': 101,
   'nuse': len(vertices),   
   'inuse': np.ones(len(vertices), dtype=np.int32),
   'vertno': np.arange(len(vertices)),
   'nvertex': len(vertices)
}
src = mne.source_space.SourceSpaces([src_dict])

# Calcul du forward
raw = mne.io.read_raw_ctf('C:/Users/wbou2/Documents/brainstorm_db/TutorialIntroduction/sample_introduction/sample_introduction/data/S01_AEF_20131218_01_600Hz.ds', preload=True)
sphere = mne.make_sphere_model(r0='auto', head_radius=0.09, info=raw.info)

fwd = mne.make_forward_solution(
   info=raw.info,
   src=src,
   bem=sphere,
   trans=None,
   meg=True, 
   eeg=False
)

# Récupération du leadfield
leadfield = fwd['sol']['data']

ds directory : C:\Users\wbou2\Documents\brainstorm_db\TutorialIntroduction\sample_introduction\sample_introduction\data\S01_AEF_20131218_01_600Hz.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       2.51   74.26    0.00 mm <->    2.51   74.26   -0.00 mm (orig :  -56.69   50.20 -264.38 mm) diff =    0.000 mm
      -2.51  -74.26    0.00 mm <->   -2.51  -74.26   -0.00 mm (orig :   50.89  -52.31 -265.88 mm) diff =    0.000 mm
     108.63    0.00    0.00 mm <->  108.63    0.00    0.00 mm (orig :   67.41   77.68 -239.53 mm) diff =    0.000 mm
    Coordinate transformations established.
    Reading digitizer points from ['C:\\Users\\wbou2\\Documents\\brainstorm_db\\TutorialIntroduction\\sample_introduction\\sample_introduction\\data\\S01_AEF_20131218_01_600Hz.ds\\S01_20131218_01.pos']...
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
Picked positions of

In [33]:
import mne
import numpy as np
import scipy.io as sio
import os

# Définition du chemin d'accès
data_path = r'C:\Users\wbou2\Desktop\sub-CC710548\meg'

def create_meg_forward():
    # 1. Chargement des données des canaux
    channel_file = os.path.join(data_path, 'channel_vectorview306_acc1.mat')
    channel_data = sio.loadmat(channel_file)
    
    # 2. Création de l'info MNE
    n_channels = 321  # D'après le ChannelFlag que vous avez montré
    ch_names = [f'MEG{i:03d}' for i in range(1, n_channels + 1)]
    info = mne.create_info(
        ch_names=ch_names,
        sfreq=1000,  # Fréquence d'échantillonnage standard
        ch_types=['meg'] * n_channels
    )
    
    # 3. Chargement du modèle de tête (utilisation du fichier headmodel si disponible)
    try:
        head_file = os.path.join(data_path, 'headmodel_surf_os_meg.mat')
        head_data = sio.loadmat(head_file)
        # Si vous voulez utiliser ce modèle plus tard, les données sont dans head_data
    except:
        print("Utilisation d'un modèle sphérique par défaut")
    
    # 4. Création du modèle sphérique simple
    sphere = mne.make_sphere_model(r0='auto', head_radius=0.09, info=info)
    
    
    # 6. Création du source space
    src_dict = {
        'rr': vertices,          
        'tris': faces,           
        'ntri': len(faces),      
        'np': len(vertices),     
        'coord_frame': 4,        
        'type': 'surf',
        'id': 101,
        'nuse': len(vertices),   
        'inuse': np.ones(len(vertices), dtype=np.int32),
        'vertno': np.arange(len(vertices)),
        'nvertex': len(vertices)
    }
    src = mne.source_space.SourceSpaces([src_dict])
    
    # 7. Calcul du forward model
    fwd = mne.make_forward_solution(
        info=info,
        src=src,
        bem=sphere,
        trans=None,
        meg=True, 
        eeg=False
    )
    
    # 8. Sauvegarde du forward model
    output_file = os.path.join(data_path, 'meg_forward-fwd.fif')
    mne.write_forward_solution(output_file, fwd, overwrite=True)
    
    # 9. Récupération du leadfield pour utilisation ultérieure
    leadfield = fwd['sol']['data']
    
    return fwd, leadfield

# Exécution
if __name__ == "__main__":
    try:
        fwd, leadfield = create_meg_forward()
        print("Forward model créé avec succès!")
        print(f"Taille du leadfield : {leadfield.shape}")
    except Exception as e:
        print(f"Erreur lors de la création du forward model: {e}")

Erreur lors de la création du forward model: "kind must be one of ['grad', 'mag', 'ref_meg', 'eeg', 'seeg', 'dbs', 'ecog', 'eog', 'emg', 'ecg', 'resp', 'bio', 'misc', 'stim', 'exci', 'syst', 'ias', 'gof', 'dipole', 'chpi', 'fnirs_cw_amplitude', 'fnirs_fd_ac_amplitude', 'fnirs_fd_phase', 'fnirs_od', 'hbo', 'hbr', 'csd', 'temperature', 'gsr', 'eyegaze', 'pupil'], not meg"
